In [1]:
# Absolute path of project
project_path = 'C:/Users/arwin/Documents/dev/APS360-PROJECT/' # change this to your machine's

In [2]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
import subprocess
import skimage.io
from skimage.transform import resize
from PIL import Image

Load the first 1.2 seconds of the WAV file

In [3]:
# download_path = project_path + 'data/billboard_audio/' 
download_path = project_path + 'data/random_audio/' 

# df = pd.read_csv( project_path + "data/billboard_2016-2021_song_artist_cleaned_rows.csv")
df = pd.read_csv( project_path + "data/random_2016-2021_song_artist_genres_cleaned_rows.csv")


Only Run This Cell When Need To Convert All webm Files to MP3

In [4]:

for index, row in enumerate(df.itertuples(), start=1):
    try:
        song = getattr(row, 'song')
        artist = getattr(row, 'artist')

        song_dir_path = download_path + artist + "_" + song

        song_dir_path = song_dir_path + '/' + os.listdir(song_dir_path)[0]
        # print(os.path.splitext(song_dir_path))
        if os.path.exists(song_dir_path):
            command = ['ffmpeg', '-y', '-hwaccel', 'cuda', '-i', song_dir_path, os.path.splitext(song_dir_path)[0] + '.mp3']
            subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print(index, os.listdir( download_path + artist + "_" + song))
        else:
            print("not found")
        
    except Exception as e:
        print(index, e)
        continue
     

1 ['Smooth Lounge Piano_Most Beautiful.webm.mp3', 'Smooth Lounge Piano_Most Beautiful.webm.part']
2 ['Gabriel Navarro_Precioso Jesus.mp3', 'Gabriel Navarro_Precioso Jesus.webm']
3 ['Raymond Fairchild_Maggie Valley Bound.mp3', 'Raymond Fairchild_Maggie Valley Bound.webm']
4 ['Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.mp3', 'Jason Stephenson_Nighttime Mind Booster Night Time Affirmations.webm.part']
5 ['Jimmy Donn_The Company I Keep.mp3', 'Jimmy Donn_The Company I Keep.webm']
6 ['Los Morros del Valle_Besos Y Caricias (En Vivo).webm.mp3', 'Los Morros del Valle_Besos Y Caricias (En Vivo).webm.part']
7 ['Madd Marvin_I Got You.mp3', 'Madd Marvin_I Got You.webm']
8 ['Tae Retro_NoLimitRetro  Battle at Sunset !!!.webm.mp3', 'Tae Retro_NoLimitRetro  Battle at Sunset !!!.webm.part']
9 ['Georg Philipp Telemann_Ouverture-Suite in G Minor, TWV 55g4 IV Les capricieuses.mp3', 'Georg Philipp Telemann_Ouverture-Suite in G Minor, TWV 55g4 IV Les capricieuses.webm']
10 ['Maj4l_C

In [5]:
def audio_to_spec(audio_path, out_fname="temp.png", sz = (228, 684)):
    
    # 1. CONVERT AUDIO FILE TO SPEC
    song_data, song_sr = librosa.load(os.path.join(dir, audio_path), sr=None)
    # song_data = librosa.feature.melspectrogram(y=song_data, sr=song_sr, n_mels=sz[1], n_fft=1024, hop_length=512, fmax=8000)
    song_data = librosa.feature.melspectrogram(y=song_data, sr=song_sr, n_mels=256, n_fft=2048, hop_length=512*25)

    # 2. Resize for ResNet18 input dims, with height = 228, width = 228*3
    song_data = resize(song_data, sz, order=0, mode='reflect', anti_aliasing=False)

    # 3. Split image into 3 slices by width and insert into each R G B channel
    assert(song_data.shape[1] // 3 == 228)
    channel_sz = song_data.shape[1] // 3
    r, g, b = song_data[:, :channel_sz], song_data[:, channel_sz : 2 * channel_sz], song_data[:, 2 * channel_sz : 3 * channel_sz]
    
    # 4. a - LOG, b - NORM, c - REFLECT, d - INVERT
    def norm_array(X, min=0.0, max=1.0):
        X_std = (X - X.min()) / (X.max() - X.min())
        X_scaled = X_std * (max - min) + min
        return X_scaled
    
    def prepare_array(arr):
        arr = np.log(arr + 1e-9)
        arr = norm_array(arr,0, 255).astype(np.uint8)
        arr = np.flip(arr, axis=0)
        arr = 255 - arr
        return arr

    r = prepare_array(r)
    g = prepare_array(g)
    b = prepare_array(b)
    img = np.stack([r, g, b], axis=-1)


    # 5. Save to img
    skimage.io.imsave(out_fname + ".png", img)


MAIN LOOP

In [14]:
spec_path = project_path + 'data/billboard_spec/'
# spec_path = project_path + 'data/random_spec/'

overwrite_all = False
for i, dir in enumerate(os.listdir(download_path)):
    print(i, dir)
    dir = os.path.join(download_path,dir)
    
    # check if img alr exists
    img_path = os.path.join(spec_path, os.path.basename(dir) + '.png')
    if os.path.exists(img_path) and os.path.getsize(img_path) >= 1024 and not overwrite_all:
        print('exists, skipping...')
        continue
    
    for song_file in os.listdir(dir):
        if song_file.endswith('mp3'):
            try:
                # convert_and_resize_mp3(mp3_path=os.path.join(dir, song_file), img_name=song_file.replace(".mp3", ""))
                audio_to_spec(
                    audio_path=os.path.join(dir, song_file),
                    out_fname= spec_path + song_file.replace(".mp3", "")
                )
            except:
                print("error") 
    


0 2 Chainz Featuring Ariana Grande_Rule The World
exists, skipping...
1 2 Chainz Featuring Kendrick Lamar_Momma I Hit A Lick
exists, skipping...
2 2 Chainz Featuring Lil Wayne_Gotta Lotta
exists, skipping...
3 2 Chainz Featuring Travis Scott_4 AM
exists, skipping...
4 2 Chainz Featuring Travis Scott_Whip
exists, skipping...
5 2 Chainz Featuring Ty Dolla $ign, Trey Songz & Jhene Aiko_It's A Vibe
exists, skipping...
6 2 Chainz Featuring YG & Offset_Proud
exists, skipping...
7 2 Chainz x Gucci Mane x Quavo_Good Drank
exists, skipping...
8 2 Chainz, Drake & Quavo_Bigger _ You
exists, skipping...
9 2 Chainz_Watch Out
exists, skipping...
10 21 Savage & Metro Boomin Featuring Drake_Mr_ Right Now
exists, skipping...
11 21 Savage & Metro Boomin Featuring Young Nudy_Snitches & Rats
exists, skipping...
12 21 Savage & Metro Boomin Featuring Young Thug_Rich N_gga Sh_t
exists, skipping...
13 21 Savage & Metro Boomin_Brand New Draco
exists, skipping...
14 21 Savage & Metro Boomin_Glock In My Lap
exis